In [1]:
import tweepy
import pandas as pd
import nltk
import string
import os
import re
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizer
consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [2]:
spark = SparkSession.builder.master('local[*]').appName('classifier').getOrCreate()
df = spark.read.csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df=df.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df2=spark.read.csv("trainingandtestdata/testdata.manual.2009.06.14.csv")
df2=df2.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df.show(5)
df2.show(5)

+--------+----------+--------------------+------------+-----------------+--------------------+
|polarity|  tweet_id|          tweet_date|query_string|tweet_screen_name|               Tweet|
+--------+----------+--------------------+------------+-----------------+--------------------+
|       0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|  _TheSpecialOne_|@switchfoot http:...|
|       0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|    scotthamilton|is upset that he ...|
|       0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|         mattycus|@Kenichan I dived...|
|       0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|          ElleCTF|my whole body fee...|
|       0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|           Karoli|@nationwideclass ...|
+--------+----------+--------------------+------------+-----------------+--------------------+
only showing top 5 rows

+--------+--------+--------------------+------------+-----------------+--------------------+
|polarity|tweet_id|        

In [3]:
df=df[['tweet_id','query_string','Tweet','polarity']]
df2=df2[['tweet_id','query_string','Tweet','polarity']]
df.groupBy('polarity').count().orderBy('count',ascending=False).show(10,False)
df2.groupBy('polarity').count().orderBy('count',ascending=False).show(10,False)
#df=df.toPandas()
#df.head(5)

+--------+------+
|polarity|count |
+--------+------+
|0       |800000|
|4       |800000|
+--------+------+

+--------+-----+
|polarity|count|
+--------+-----+
|4       |182  |
|0       |177  |
|2       |139  |
+--------+-----+



In [4]:
from nltk.corpus.reader.wordnet import *
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StringType,DoubleType,IntegerType
wn = nltk.WordNetLemmatizer()
worddict = set(nltk.corpus.words.words())

def preprocessing(text):
    wordset_n = set(wn.lemmatize(w, NOUN) for w in word_tokenize(text.lower().strip()))
    wordset_v = set(wn.lemmatize(w, VERB) for w in wordset_n)
    wordset = set(wn.lemmatize(w, ADJ) for w in wordset_v)
    wordset = wordset & worddict
    return ' '.join(list(wordset))
brand_udf=udf(preprocessing,StringType())
df=df.withColumn('text',brand_udf(df['Tweet']))
df=df.withColumn('clean_len',F.length('text'))
df2=df2.withColumn('text',brand_udf(df2['Tweet']))
df2=df2.withColumn('clean_len',F.length('text'))
#tem=df.union(df2)
#tem.show()

In [5]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
#tem = tokenizer.transform(tem)
df=tokenizer.transform(df)
df2=tokenizer.transform(df2)

In [6]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
#model = count.fit(tem)
#tem = model.transform(tem)
#tem.show()
model1=count.fit(df)
df=model1.transform(df)
model2=count.fit(df2)
df2=model2.transform(df2)

In [7]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(tem)
#tem = idfModel.transform(tem)
#tem = tem.withColumn('polarity', tem['polarity'].cast(IntegerType()))
idfModel = idf.fit(df)
df = idfModel.transform(df)
df = df.withColumn('polarity', df['polarity'].cast(IntegerType()))
idfModel = idf.fit(df2)
df2 = idfModel.transform(df2)
df2 = df2.withColumn('polarity', df2['polarity'].cast(IntegerType()))
trainingDF,testDF = df.randomSplit([0.7, 0.3])

In [8]:
from pyspark.ml.classification import RandomForestClassifier
rf_classifier=RandomForestClassifier(labelCol='polarity',numTrees=50).fit(trainingDF.select(['text','words','rawFeatures','features','polarity']))
rf_predictions=rf_classifier.transform(testDF.select(['text','words','rawFeatures','features','polarity']))

In [9]:
rf_predictions.show(10)

Py4JJavaError: An error occurred while calling o460.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 1 times, most recent failure: Lost task 0.0 in stage 38.0 (TID 167) (192.168.1.227 executor driver): java.io.IOException: Failed to delete original file 'C:\Users\Sajin.LAPTOP-RE0DL8PH\AppData\Local\Temp\spark-8935d2c5-a375-429e-9094-5b244d1df78f\broadcast5379028567372557823' after copy to 'C:\Users\Sajin.LAPTOP-RE0DL8PH\AppData\Local\Temp\blockmgr-c1994c25-a717-4fdc-976b-25b2e92cb905\08\broadcast_16_python'
	at org.apache.commons.io.FileUtils.moveFile(FileUtils.java:2276)
	at org.apache.spark.storage.DiskStore.moveFileToBlock(DiskStore.scala:133)
	at org.apache.spark.storage.BlockManager$TempFileBasedBlockStoreUpdater.saveToDiskStore(BlockManager.scala:478)
	at org.apache.spark.storage.BlockManager$BlockStoreUpdater.$anonfun$save$1(BlockManager.scala:397)
	at org.apache.spark.storage.BlockManager.org$apache$spark$storage$

In [ ]:
# Follow every follower of the authenticated user
timeline = api.home_timeline()
data = pd.DataFrame(data=[tweet.text for tweet in timeline], columns=['Tweet'])